# Step 1: Extract Audio from Video

In [ ]:
from moviepy.editor import VideoFileClip
clip = VideoFileClip("RAM.mp4")
clip.audio.write_audiofile("RAM.wav")


MoviePy - Writing audio in RAM.wav


MoviePy - Done.


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!apt-get update && apt-get install -y ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-d_zj4pfp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-d_zj4pfp
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803669 sha256=c7cce367296096cd0568565223f9fac060ec47e60eef2561eb9d008e539f7815
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbgef7sx/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Looking in indexes: https://download.pytorch.org/whl/cu118
Get:1 https://devel

#Step 2: Extract Text from Audio

In [ ]:
import whisper

# Load the Whisper model
model = whisper.load_model("medium")

# Transcribe the uploaded audio file with word timestamps enabled
result = model.transcribe("RAM.wav", word_timestamps=True)

100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 93.2MiB/s]
  checkpoint = torch.load(fp, map_location=device)



In [ ]:
result

{'text': " We're in war with Rick and Morty. You'll want to make a stop at the White House. I have plans at the Pentagon, Ronnie. It's in my contract. I mean, you need to call a press conference. Israel and Palestine have announced a permanent ceasefire. What? They signed something called the Pretty Obvious If You Think About It accord. Apparently, an anonymous American diplomat took them to a Star Wars cantina where they smoked perspective-enhancing alien pheromones through a laser hookah. I still say it has to be Putin. It was Rick and Morty, you fucking dunce! But you're getting the credit, sir. Your approval rating just hit 100%. Why would Rick and Morty want that? Because they're assholes! I'm flattered, but it takes a whole team to achieve what we did today. Thank you. Well? They haven't been back home, sir. We've got every possible location under surveillance. God damn it! They're right here! They're here, god damn it! Cobert! Cobert! Everybody relax. Just waiting for you in the

In [ ]:
# Initialize an empty dictionary to store words and their corresponding start and end times
text_timings = {}

# Iterate over the result to get the segments
for segment in result['segments']:
    # Split the segment text into words and assign their respective start and end times
    for word_info in segment['words']:
        word = word_info['word']
        start_time = word_info['start']
        end_time = word_info['end']

        # If the word already exists in the dictionary, append the new timings to the list
        if word in text_timings:
            text_timings[word].append({"start": start_time, "end": end_time})
        else:
            # If it's the first occurrence of the word, initialize it with a list of timings
            text_timings[word] = [{"start": start_time, "end": end_time}]

# Print the resulting dictionary
print(text_timings)



{" We're": [{'start': 0.0, 'end': 0.44}], ' in': [{'start': 0.44, 'end': 0.6}, {'start': 5.06, 'end': 5.16}, {'start': 48.46, 'end': 48.54}, {'start': 57.14, 'end': 57.3}, {'start': 107.6, 'end': 107.74}, {'start': 134.02, 'end': 134.18}, {'start': 139.96, 'end': 140.14}, {'start': 148.54, 'end': 148.66}], ' war': [{'start': 0.6, 'end': 0.82}], ' with': [{'start': 0.82, 'end': 1.06}, {'start': 140.56, 'end': 140.84}, {'start': 153.88, 'end': 154.12}], ' Rick': [{'start': 1.06, 'end': 1.26}, {'start': 24.92, 'end': 25.1}, {'start': 29.92, 'end': 30.08}, {'start': 155.24, 'end': 155.42}], ' and': [{'start': 1.26, 'end': 1.4}, {'start': 8.16, 'end': 8.4}, {'start': 25.1, 'end': 25.24}, {'start': 30.08, 'end': 30.2}, {'start': 55.18, 'end': 55.32}, {'start': 64.36, 'end': 64.62}, {'start': 87.54, 'end': 87.66}, {'start': 113.72, 'end': 114.0}, {'start': 118.58, 'end': 118.86}, {'start': 155.42, 'end': 155.54}], ' Morty.': [{'start': 1.4, 'end': 1.62}], " You'll": [{'start': 1.72, 'end': 1.

# Step 3: Classify Text for Profanity/Insult/Hate Speech

In [ ]:
# prompt: read text file and store each line as text

def read_text_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            return [line.strip() for line in lines]  # Remove leading/trailing whitespace
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

# Example usage
file_path = 'en.txt'
text_lines = read_text_file(file_path)

bad_words = []

if text_lines:
    for line in text_lines:
        bad_words.append(line)


In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

# Define a threshold for similarity (e.g., 80% match)
threshold = 81

# Initialize a dictionary to store offensive words and their corresponding start and end times
offensive_words_timings = {}

# Iterate through each word and its timings in the text_timings dictionary
for word, timings_list in text_timings.items():
    # Iterate over the list of timings for the current word
    for timings in timings_list:
        for bad_word in bad_words:
            # Calculate the similarity between the word and each bad word
            similarity = fuzz.ratio(word.lower(), bad_word.lower())
            if similarity >= threshold:  # If the similarity score is above the threshold
                # Store the offensive word and its timestamps in the dictionary
                if word not in offensive_words_timings:
                    offensive_words_timings[word] = []
                offensive_words_timings[word].append({
                    "start": timings['start'],
                    "end": timings['end'],
                    "similarity": similarity
                })

# Print the dictionary of offensive words and their timings
offensive_words_timings


  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



{' fucking': [{'start': 25.62, 'end': 25.98, 'similarity': 86},
  {'start': 25.62, 'end': 25.98, 'similarity': 93},
  {'start': 25.62, 'end': 25.98, 'similarity': 88},
  {'start': 144.22, 'end': 144.56, 'similarity': 86},
  {'start': 144.22, 'end': 144.56, 'similarity': 93},
  {'start': 144.22, 'end': 144.56, 'similarity': 88},
  {'start': 156.4, 'end': 156.62, 'similarity': 86},
  {'start': 156.4, 'end': 156.62, 'similarity': 93},
  {'start': 156.4, 'end': 156.62, 'similarity': 88}],
 ' hit': [{'start': 28.54, 'end': 28.7, 'similarity': 89}],
 ' assholes!': [{'start': 31.5, 'end': 32.26, 'similarity': 82},
  {'start': 31.5, 'end': 32.26, 'similarity': 89}],
 ' damn': [{'start': 43.2, 'end': 43.4, 'similarity': 89},
  {'start': 45.2, 'end': 45.34, 'similarity': 89}],
 '-ass': [{'start': 70.24, 'end': 70.4, 'similarity': 86}],
 ' piss': [{'start': 80.98, 'end': 81.22, 'similarity': 89}],
 ' motherfucker.': [{'start': 148.82, 'end': 149.28, 'similarity': 89},
  {'start': 148.82, 'end': 1

# Step 4: Censor Audio

In [ ]:
!pip install pydub
!pip install simpleaudio
!brew install ffmpeg
!sudo apt update
!sudo apt install ffmpeg


/bin/bash: line 1: brew: command not found
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'htt

In [ ]:
from pydub import AudioSegment
from pydub.generators import Sine

# Load the original audio
audio = AudioSegment.from_file("RAM.wav")

# Function to beep a section of the audio
def beep_audio_section(audio, start_time, end_time):
    # Convert the start and end times from seconds to milliseconds
    start_ms = start_time * 1000
    end_ms = end_time * 1000
    # Generate a beep sound (1 second duration at 1000 Hz frequency)
    beep = Sine(1000).to_audio_segment(duration=500)  # 500ms beep sound
    # Replace the section with the beep sound
    audio = audio[:start_ms] + beep + audio[end_ms:]
    return audio

# Iterate over offensive words and beep the respective sections
for word, timings_list in offensive_words_timings.items():
    # Iterate through each timing in the list of timings for the word
    for timings in timings_list:
        timestamp_start = timings['start']
        timestamp_end = timings['end']
        print(f"Beeping offensive word: {word.strip()}, Start: {timestamp_start}, End: {timestamp_end}")

        # Beep the section of audio
        audio = beep_audio_section(audio, timestamp_start, timestamp_end)

# Export the modified audio (censored with beeps)
audio.export("censored_audio_with_beeps.wav", format="wav")


Beeping offensive word: fucking, Start: 25.62, End: 25.98
Beeping offensive word: fucking, Start: 25.62, End: 25.98
Beeping offensive word: fucking, Start: 25.62, End: 25.98
Beeping offensive word: fucking, Start: 144.22, End: 144.56
Beeping offensive word: fucking, Start: 144.22, End: 144.56
Beeping offensive word: fucking, Start: 144.22, End: 144.56
Beeping offensive word: fucking, Start: 156.4, End: 156.62
Beeping offensive word: fucking, Start: 156.4, End: 156.62
Beeping offensive word: fucking, Start: 156.4, End: 156.62
Beeping offensive word: hit, Start: 28.54, End: 28.7
Beeping offensive word: assholes!, Start: 31.5, End: 32.26
Beeping offensive word: assholes!, Start: 31.5, End: 32.26
Beeping offensive word: damn, Start: 43.2, End: 43.4
Beeping offensive word: damn, Start: 45.2, End: 45.34
Beeping offensive word: -ass, Start: 70.24, End: 70.4
Beeping offensive word: piss, Start: 80.98, End: 81.22
Beeping offensive word: motherfucker., Start: 148.82, End: 149.28
Beeping offensiv

<_io.BufferedRandom name='censored_audio_with_beeps.wav'>

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

# Load the video and the modified audio with beeps
video = VideoFileClip("RAM.mp4")
modified_audio = AudioFileClip("censored_audio_with_beeps.wav")

# Ensure the audio length matches the video
video_duration = video.duration
modified_audio = modified_audio.subclip(0, video_duration)  # Trim audio to match video duration

# Set the new audio to the video
video = video.set_audio(modified_audio)

# Export the final video with censored audio
video.write_videofile("final_video_with_censored_audio.mp4", codec="libx264", audio_codec="aac")


Moviepy - Building video final_video_with_censored_audio.mp4.
MoviePy - Writing audio in final_video_with_censored_audioTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_video_with_censored_audio.mp4



t: 100%|█████████▉| 4061/4062 [02:39<00:00, 35.72it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file RAM.mp4, 2764800 bytes wanted but 0 bytes read,at frame 4060/4062, at time 169.34/169.39 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready final_video_with_censored_audio.mp4


In [ ]:
from transformers import pipeline

# Initialize the classifier
classifier = pipeline("text-classification", model="unitary/toxic-bert")

# Input text to be classified
input_text = result["text"]

# Split the text into two parts
text_length = len(input_text)
half_length = text_length // 2
part1 = input_text[:half_length]
part2 = input_text[half_length:]

# Get classification results for both parts
result_part1 = classifier(part1)
result_part2 = classifier(part2)

# Extract the toxicity scores (assuming the label is 'LABEL_1' for toxic)
score_part1 = result_part1[0]['score']
score_part2 = result_part2[0]['score']

# Calculate the average score
average_score = (score_part1 + score_part2) / 2

# Print the individual scores and average score
print(f"Score for Part 1: {score_part1}")
print(f"Score for Part 2: {score_part2}")
print(f"Average toxic Score: {average_score}")


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Score for Part 1: 0.9257194995880127
Score for Part 2: 0.9560049772262573
Average toxic Score: 0.940862238407135


In [ ]:
# Calculate the number of offensive words based on the values in the dictionary
num_offensive_words = sum(1 for _ in offensive_words_timings.values())

# Calculate the total number of words in the result text
total_words = len(result["text"].split())

# Calculate the offensive score as the ratio of offensive words to total words
offensive_score = num_offensive_words / total_words if total_words != 0 else 0

non_offensive_score = 1 - offensive_score
# Print the offensive score
print(f"offensive_score: {offensive_score}")
print(f"non_offensive_score: {non_offensive_score}")

offensive_score: 0.012403100775193798
non_offensive_score: 0.9875968992248062
